<div class="usecase-section-header">Ideal Green Laneway Locations: Melbourne City</div>

<div class="usecase-authors"><b>Authored by: </b>Alison Collins</div>

<div class="usecase-duration"><b>Duration:</b>60 mins</div>
<div class="usecase-level"><b>Level: </b>beginner</div>
<div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>

<div class="usecase-section-header">Scenario</div>
1. As a city planner, I want to identify the laneways that could most benefit from participating in the Green Your Laneway program.
2. As a building manager or owner of a residence, I want to visualise the potential of laneway greening for energy efficiency.

<div class="usecase-section-header">Learning Objectives</div>
At the end of this use case you will be able to:
* Work with spatial databases using Geopandas
* Visualise spatial data on an interactive map



In [2]:
import requests
import pandas as pd
import os

def fetch_data(base_url, dataset, api_key, num_records=99, offset=0):
    all_records = []
    max_offset = 9900  # Maximum number of requests

    while True:
        # maximum limit check
        if offset > max_offset:
            break

        # Create API request URL
        filters = f'{dataset}/records?limit={num_records}&offset={offset}'
        url = f'{base_url}{filters}&apikey={api_key}'

        # Start request
        try:
            result = requests.get(url, timeout=10)
            result.raise_for_status()
            records = result.json().get('results')
        except requests.exceptions.RequestException as e:
            raise Exception(f"API request failed: {e}")
        if records is None:
            break
        all_records.extend(records)
        if len(records) < num_records:
            break

        # next cycle offset
        offset += num_records

    # DataFrame all data
    df = pd.DataFrame(all_records)
    return df

API_KEY = os.environ.get('MELBOURNE_API_KEY', input("Please enter your API key: "))
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'

In [4]:
# data set name
SENSOR_DATASET = 'laneways-with-greening-potential'

df = fetch_data(BASE_URL, SENSOR_DATASET, API_KEY)

df.head(3)

,geo_point_2d,geo_shape,segid_1,mapbase_mc,insol_scor,verticalsu,segid,driveways,parklane,objectid,...,farmlane,walls,farm_rank,vert_rank,fores_rank,forest,park_rank,binsperct,mapbase_1,wsud
0,"{'lon': 144.9623447558369, 'lat': -37.81980050...","{'type': 'Feature', 'geometry': {'coordinates'...",10714,10714.0,6.492595,0.000000,10714,1,53.830526,1,...,58.518386,Probably Unsuitable,Highest Potential,Lowest potential,Lowest potential,0.000000,Highest potential,0.0,Banana Alley,Probably Unsuitable
1,"{'lon': 144.97170834375964, 'lat': -37.8109763...","{'type': 'Feature', 'geometry': {'coordinates'...",20229,20229.0,2.215613,0.000000,20229,2,0.000000,5,...,0.000000,Probably Unsuitable,Lowest potential,Lowest potential,Good potential,49.153357,Lowest potential,60.0,Harwood Place,Consider
2,"{'lon': 144.97244046374837, 'lat': -37.8111115...","{'type': 'Feature', 'geometry': {'coordinates'...",20230,20230.0,2.750935,63.214997,20230,0,0.000000,6,...,0.000000,Consider,Lowest potential,Highest potential,Lowest potential,0.000000,Lowest potential,60.0,Turnbull Alley,Consider
